<a href="https://colab.research.google.com/github/namita0210/ResearchOnImageCaptioning/blob/main/imageCaptionGeneration/captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as numpy

In [5]:
path = '/content/drive/MyDrive/flickr8kDataset'